# Análise das podas do AQE

Nesta análise vamos experimentar diferentes abordagens de poda para o AQE, de forma a evitar que a expansão seja muito longa e acabe por prejudicar as consultas do Elasticsearch.

## Carregando libs

In [ ]:
from itertools import product
import json
import yaml
import pandas as pd
import plotly.express as px

from utils.utils import get_expanded_queries, make_elasticsearch_new_aqe_queries,\
    create_new_expanded_queries, create_new_aqe_validation_dataset, create_new_aqe_metrics,\
    expanded_with_aqe_boost_order, expanded_with_aqe_order

## Carregando as configurações e bases dados

In [ ]:
with open("../conf/config.yaml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

In [ ]:
with open("../../dados/regis/regis_queries.json", 'r') as regis_file:
    regis_queries = json.load(regis_file)

In [ ]:
regis_queries = get_expanded_queries(regis_queries)
regis_queries[:2]

In [ ]:
ground_truth = pd.read_csv(
    "../../dados/regis/regis_ground_truth.csv"
).rename(
    columns={"relevance": "relevance_ground_truth"}
)
ground_truth.head()

## Criando queries com podas baseadas nos fatores de boosting do AQE

Aqui vamos experimentar uma poda com diferentes quantidades de termos utilizando os pesos já estabelecidos pelo AQE.

In [ ]:
all_expanded_queries = list()
for query in regis_queries:
    new_expanded_queries = create_new_expanded_queries(query["expanded_query"], expansion=expanded_with_aqe_boost_order)
    for num_termos, new_expanded_query in new_expanded_queries:
        q = query.copy()
        q["expanded_query"] = new_expanded_query
        q["num_termos"] = num_termos
        all_expanded_queries.append(q)
all_expanded_queries[:2]

### Realizando consultas no Elasticsearch

Em posse das queries que utilizam diferentes quantidades de termos com boosting do elastic search vamos criar o dataset de validação, o qual possui informações do ground truth da base de dados REGIS.

In [ ]:
ranking_result_df = make_elasticsearch_new_aqe_queries(
    all_expanded_queries,
    cfg,
    num_docs=24
)
ranking_result_df.head()

In [ ]:
validation_dataset = create_new_aqe_validation_dataset(ranking_result_df, ground_truth)
validation_dataset.head()

### Análise das consultas no Elasticsearch

Agora vamos criar as métricas para cada base de dados e quantidade de termos derivados e visualizar os resultados.

#### Criando métricas

In [ ]:
metrics_df = create_new_aqe_metrics(validation_dataset)
metrics_df.head()

#### Avaliando métricas

Vamos agora avaliar as métricas. Vamos utilizar as seguintes métricas:

* ndcg - Normalized Discounted Cumulative Gain
* map - Mean Average Precision
* eval_prop - Proporção de documentos avaliados

Vejamos qual a melhor quantidade de termos derivados para cada query:

In [ ]:
data_viz = metrics_df.groupby(
    "query_id"
).agg({
    "ndcg@24": "max"
}).reset_index(
).merge(
    metrics_df, how="left", on=["query_id", "ndcg@24"]
).sort_values(
    ["query_id", "num_termos"]
).drop_duplicates(
    subset="query_id", keep="first"
)
data_viz.head()

fig = px.scatter(
    data_viz,
    x="num_termos",
    y="ndcg@24",
    labels={
        "num_termos": "Número de termos",
        "ndcg@24": "NDCG@24",
    },
    hover_data=["query_id", "num_termos", "ndcg@24"],
    title="Melhor número de termos por query",
    marginal_x="histogram"
)
fig.show()

Podemos ver que a maior concentração está abaixo dos 5 termos derivados.

Vejamos a média para cada número de termos derivados:

In [ ]:
queries_boosts_prod = pd.DataFrame(
    product(metrics_df["query_id"].unique(), metrics_df["num_termos"].unique()),
    columns=["query_id", "num_termos"]
)

data_viz = queries_boosts_prod.merge(
    metrics_df, on=["query_id", "num_termos"], how="left"
).fillna(
    method="ffill"
).groupby(
    "num_termos"
).agg(
    mean_ndcg = ("ndcg@24", "mean")
).reset_index()

fig = px.line(
    data_viz,
    x="num_termos",
    y="mean_ndcg",
    labels={
        "num_termos": "Número de termos",
        "mean_ndcg": "NDCG@24 médio",
    },
    markers=True,
    title="NDCG@24 médio para cada número de termos"
)
fig.show()

Podemos ver que no geral, utilizar 5 termos derivados traz o melhor resultado, o qual é melhor que o Elasticsearch puro.

Vejamos como fica a distribuição dos NDCGs@24 ao utilizar o limiar de poda de 5 termos:

In [ ]:
metrics_df_poda = metrics_df.query(
    "num_termos <= 5"
).sort_values(
    ["query_id", "num_termos", "ndcg@24"]
).groupby(
    "query_id"
).last()

data_viz = metrics_df.groupby(
    "query_id"
).agg({
    "ndcg@24": "max"
}).merge(
    metrics_df_poda,
    on="query_id",
    suffixes=(" max", ""),
    how="left"
).reset_index(
).melt(
    id_vars=["query_id"],
    value_vars=["ndcg@24 max", "ndcg@24"],
    var_name="metric"
).sort_values(
    ["metric", "value"], ascending=[True, False]
)

fig = px.bar(
    data_viz,
    x="query_id",
    y="value",
    color="metric",
    barmode='group',
    labels={
        "query_id": "Query ID",
        "value": "NDCG@24",
    },
)
fig.show()

Podemos ver que mais de metade das queries possuem NDCG@24 acima de 0,8.
Podemos ver também que apenas quatro das 32 queries tem uma diferença substancial entre o NDCG@24 com cinco termos expandidos e o máximo. São elas: Q19, Q17, Q23 e Q34 .

Vejamos quais as melhores quantidades de termos para essas queries:

In [ ]:
queries = ["Q23", "Q34", "Q19", "Q17"]

metrics_df.query(
    "query_id.isin(@queries)"
).groupby(
    "query_id"
).agg({
    "ndcg@24": "max"
}).reset_index(
).merge(
    metrics_df, how="left", on=["query_id", "ndcg@24"]
).sort_values(
    ["query_id", "num_termos"]
).drop_duplicates(
    subset="query_id", keep="first"
)

Podemos ver que exceto a Q17 todos os valores foram próximos de 5. A Q17 parece ser um caso atípico de que algum termo que era considerado de baixa relevância trouxe bons resultados.

## Criando queries com podas baseadas na ordem do AQE

Aqui vamos experimentar uma poda com diferentes quantidades de termos utilizando a ordem do AQE. A motivação desse experimento é devido ao padrão de fatores de boostings usados, os quais trazem pesos maiores no início e depois vão se misturando, que indica que algum processamento foi realizado.

In [ ]:
all_expanded_queries = list()
for query in regis_queries:
    new_expanded_queries = create_new_expanded_queries(query["expanded_query"], expansion=expanded_with_aqe_order)
    for num_termos, new_expanded_query in new_expanded_queries:
        q = query.copy()
        q["expanded_query"] = new_expanded_query
        q["num_termos"] = num_termos
        all_expanded_queries.append(q)
all_expanded_queries[:2]

### Realizando consultas no Elasticsearch

Em posse das queries que utilizam diferentes quantidades de termos com boosting do elastic search vamos criar o dataset de validação, o qual possui informações do ground truth da base de dados REGIS.

In [ ]:
ranking_result_df = make_elasticsearch_new_aqe_queries(
    all_expanded_queries,
    cfg,
    num_docs=24
)
ranking_result_df.head()

In [ ]:
validation_dataset = create_new_aqe_validation_dataset(ranking_result_df, ground_truth)
validation_dataset.head()

### Análise das consultas no Elasticsearch

Agora vamos criar as métricas para cada base de dados e quantidade de termos derivados e visualizar os resultados.

#### Criando métricas

In [ ]:
metrics_df = create_new_aqe_metrics(validation_dataset)
metrics_df.head()

#### Avaliando métricas

Vamos agora avaliar as métricas. Vamos utilizar as seguintes métricas:

* ndcg - Normalized Discounted Cumulative Gain
* map - Mean Average Precision
* eval_prop - Proporção de documentos avaliados

Vejamos qual a melhor quantidade de termos derivados para cada query:

In [ ]:
data_viz = metrics_df.groupby(
    "query_id"
).agg({
    "ndcg@24": "max"
}).reset_index(
).merge(
    metrics_df, how="left", on=["query_id", "ndcg@24"]
).sort_values(
    ["query_id", "num_termos"]
).drop_duplicates(
    subset="query_id", keep="first"
)
data_viz.head()

fig = px.scatter(
    data_viz,
    x="num_termos",
    y="ndcg@24",
    labels={
        "num_termos": "Número de termos",
        "ndcg@24": "NDCG@24",
    },
    hover_data=["query_id", "num_termos", "ndcg@24"],
    title="Melhor número de termos por query",
    marginal_x="histogram"
)
fig.show()

Podemos ver que a maior concentração está abaixo dos 5 termos derivados.

Vejamos a média para cada número de termos derivados:

In [ ]:
queries_boosts_prod = pd.DataFrame(
    product(metrics_df["query_id"].unique(), metrics_df["num_termos"].unique()),
    columns=["query_id", "num_termos"]
)

data_viz = queries_boosts_prod.merge(
    metrics_df, on=["query_id", "num_termos"], how="left"
).fillna(
    method="ffill"
).groupby(
    "num_termos"
).agg(
    mean_ndcg = ("ndcg@24", "mean")
).reset_index()

fig = px.line(
    data_viz,
    x="num_termos",
    y="mean_ndcg",
    labels={
        "num_termos": "Número de termos",
        "mean_ndcg": "NDCG@24 médio",
    },
    markers=True,
    title="NDCG@24 médio para cada número de termos"
)
fig.show()

Podemos ver que utilizar os primeiros 3 termos derivados traz o melhor resultado, o qual é melhor que o Elasticsearch puro.

Vejamos como fica a distribuição dos NDCGs@24 ao utilizar o limiar de poda de 3 termos:

In [ ]:
metrics_df_poda = metrics_df.query(
    "num_termos <= 3"
).sort_values(
    ["query_id", "num_termos", "ndcg@24"]
).groupby(
    "query_id"
).last()

data_viz = metrics_df.groupby(
    "query_id"
).agg({
    "ndcg@24": "max"
}).merge(
    metrics_df_poda,
    on="query_id",
    suffixes=(" max", ""),
    how="left"
).reset_index(
).melt(
    id_vars=["query_id"],
    value_vars=["ndcg@24 max", "ndcg@24"],
    var_name="metric"
).sort_values(
    ["metric", "value"], ascending=[True, False]
)

fig = px.bar(
    data_viz,
    x="query_id",
    y="value",
    color="metric",
    barmode='group',
    labels={
        "query_id": "Query ID",
        "value": "NDCG@24",
    },
)
fig.show()

Podemos ver que metade das queries possuem NDCG@24 acima de 0,8.
Podemos ver também que apenas quatro das 32 queries tem uma diferença substancial entre o NDCG@24 com os três primeiros termos expandidos e o máximo. São elas: Q17, Q34, Q19 e Q3.

Vejamos quais as melhores quantidades de termos para essas queries:

In [ ]:
queries = ["Q17", "Q34", "Q19", "Q3"]

metrics_df.query(
    "query_id.isin(@queries)"
).groupby(
    "query_id"
).agg({
    "ndcg@24": "max"
}).reset_index(
).merge(
    metrics_df, how="left", on=["query_id", "ndcg@24"]
).sort_values(
    ["query_id", "num_termos"]
).drop_duplicates(
    subset="query_id", keep="first"
)

Podemos ver que exceto a Q17 e Q3 todos foram próximos de 3 termos. A Q17 e Q3 parecem ser casos atípicos, em que algum termo que era considerado de baixa relevância trouxe bons resultados.

## Conclusão

Nesta análise vimos que a poda dos métodos do AQE ajudam a melhorar as métricas de performance. Vimos duas abordagens de poda: utilizar a ordem dos fatores de boosting e a ordem provinda do AQE. Em ambos, os melhores resultados de NDCG@24 foram na casa de 0.76, sendo ambos superiores ao Elasticsearch puro, mas a ordem do AQE trouxe um resultado ligeiramente melhor. Apesar disso, consideramos que utilizar a ordem dos fatores de boosting é o mecanismo de poda mais confiável, pois a variação do NDCG@24 em maiores número de termos é bem menor, tornando um método de poda mais estável. Além disso, podando com até 5 termos, as queries que trazem maiores diferenças apresentam número de termos mais próximos. Logo, concluímos que utilizar a ordem do fator de boosting com até cinco termos é o método de poda mais razoável para o caso geral.